In [16]:
import pandas as pd
import numpy as np

# Set options to show dfs in full
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

import requests
from bs4 import BeautifulSoup

In [4]:
# Import the outbreak data
# Globalnews source
glob = pd.read_csv('../data/BC/globalnews_scrape_2.csv')

# Nora Loreto source
nora = pd.read_csv('../data/BC/Deaths in Residential Care in Canada by facility - Institutions.csv', skiprows=[0,1])


In [5]:
nora.info()
nora.head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645 entries, 0 to 644
Data columns (total 11 columns):
Facility                 633 non-null object
Location                 636 non-null object
Prov.                    631 non-null object
Number of dead           638 non-null float64
Number of beds (appx)    203 non-null float64
%                        520 non-null object
Public or private        256 non-null object
Owners                   126 non-null object
Union                    51 non-null object
Unnamed: 9               18 non-null object
Unnamed: 10              9 non-null object
dtypes: float64(2), object(9)
memory usage: 55.5+ KB


<bound method NDFrame.head of                                                            Facility  \
0    CHSLD Vigi Mont-Royal*                                           
1    Centre d'hébergement Sainte-Dorothée                             
2    Centre d'hébergement Notre-Dame-de-la-Merci                      
3    CHSLD Laurendeau                                                 
4    Centre d'hébergement Champlain–Marie-Victorin                    
5    CHSLD Yvon-Brunet                                                
6    CHSLD Vigi Dollard-des-Ormeaux                                   
7    CHSLD Saint-Jude                                                 
8    Orchard Villa LTC                                                
9    CHSLD Pierre-Joseph-Triest                                       
10   Camilla Care Community                                           
11   CHSLD Grace Dart                                                 
12   CHSLD Angelica                            

In [7]:
BC_cases = nora.loc[nora['Prov.'] == 'BC']
BC_cases.info()
BC_cases

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 82 to 630
Data columns (total 11 columns):
Facility                 22 non-null object
Location                 22 non-null object
Prov.                    22 non-null object
Number of dead           22 non-null float64
Number of beds (appx)    3 non-null float64
%                        17 non-null object
Public or private        14 non-null object
Owners                   3 non-null object
Union                    0 non-null object
Unnamed: 9               1 non-null object
Unnamed: 10              1 non-null object
dtypes: float64(2), object(9)
memory usage: 2.1+ KB


,Facility,Location,Prov.,Number of dead,Number of beds (appx),%,Public or private,Owners,Union,Unnamed: 9,Unnamed: 10
82,Langley Lodge,Langley,BC,26.0,121.0,21.49%,"Private, not for profit",NaN,NaN,NaN,NaN
117,Lynn Valley Care Centre,North Vancouver,BC,20.0,204.0,9.80%,Private,NaN,NaN,NaN,NaN
163,Holy Family Hospital long term care unit,Vancouver,BC,15.0,126.0,NaN,NaN,Providence Health care,NaN,NaN,NaN
182,Haro Park,Vancouver,BC,13.0,NaN,#DIV/0!,"Private, not for profit",NaN,NaN,NaN,NaN
194,Royal Arch Masonic Home,Vancouver,BC,12.0,NaN,#DIV/0!,"Private, not for profit",NaN,NaN,NaN,NaN
244,Lions Gate Hospital (Unit 6 East),Vancouver,BC,8.0,NaN,NaN,Public,NaN,NaN,NaN,NaN
292,Berkley Care Centre,North Vancouver,BC,5.0,NaN,#DIV/0!,Public,NaN,NaN,NaN,NaN
324,Dufferin Care Centre,Coquitlam,BC,4.0,NaN,#DIV/0!,"Private, for profit",NaN,NaN,NaN,NaN
355,Four LTC homes in Vancouver - July 6,Vancouver,BC,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,Amica Edgemont Village,North Vancouver,BC,3.0,NaN,#DIV/0!,"Private, for profit",BayBridge Seniors Housing,NaN,https://seniorhousingnews.com/2015/09/02/baybridge-acquires-amica-for-578-million-in-latest-canadian-mega-deal/,NaN


In [9]:
BC_cases['cleaned_name'] = BC_cases['Facility'].str.lower()

/Users/mds/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
glob.info()
glob
glob['cleaned_name'] = glob['facility_name'].str.lower()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 3 columns):
facility_name    58 non-null object
num_cases        58 non-null int64
num_deaths       58 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.4+ KB


In [14]:
BC_cases['cleaned_name'].isin(glob['cleaned_name']).value_counts()

False    12
True     10
Name: cleaned_name, dtype: int64

In [12]:
# # Homes in the global data not in the Nora data
# a = set(BC_cases['cleaned_name'])
# b = set(glob['cleaned_name'])

# def returnNotMatches(a, b):
#     return [x for x in b if x not in a]

# missing = returnNotMatches(a, b)
# len(missing)
# # missing

48

In [15]:
# Homes (12) in the Nora data not in the Global data
a = set(glob['cleaned_name'])
b = set(BC_cases['cleaned_name'])

def returnNotMatches(a, b):
    return [x for x in b if x not in a]

missing = returnNotMatches(a, b)
missing

['amica edgemont village',
 'holy family hospital long term care unit',
 'shaughnessy care centre',
 'worthington longterm care facility',
 'mission institution*',
 'residence at clayton heights',
 'lions gate hospital (unit 6 east)',
 'swedish assisted living',
 'four ltc homes in vancouver - july 6',
 'ridge meadows hospital (rehab and unit 3w)',
 'haro park',
 'chartwell willow long-term care ']

## Manual matching of Nora's data to Global data
### Matched
- 'amica edgemont village' - Amica Edgemont
- 'holy family hospital long term care unit' - Holy Family Hospital
- 'shaughnessy care centre' - Shaughnessy Care Center
- 'worthington longterm care facility' - Worthington Pavilion LCTF OR Worthington Pavilion / MSA Rehab
- 'residence at clayton heights', Worthington Pavilion / MSA Rehab
- 'lions gate hospital (unit 6 east)', Lions Gate Hospital
- 'swedish assisted living', Swedish Canadian Manor, assisted living
- 'ridge meadows hospital (rehab and unit 3w)', Ridge Meadows Hospital (3 entries
- 'haro park', Haro Park Centre
- 'chartwell willow long-term care', Chartwell Willow Long-term care¬†

### Not Matched
- 'mission institution*' - correctional facility
- 'four ltc homes in vancouver - july 6', 

## Scrape BC health authority websites

In [17]:
# Northern
# Need to scrape facilities table instead of public exposures

re = requests.get('https://www.northernhealth.ca/health-topics/current-outbreaks')
s = BeautifulSoup(re.text,'html5lib')
table =s.find_all('td')

d = ['city', 'facility', 'outbreaktype', 'datedeclared']
outbreak = {}
for each,i in zip(table,d):
    outbreak[i] = each.text.strip()
    
northern = pd.DataFrame(outbreak, index = [0])
northern['health_authority'] = "northern"

northern.datedeclared = northern.datedeclared.str.replace('\n\n\t\t\t', '-')

northern

In [22]:
# VIHA
# Outbreaks that are ended are removed
vi = requests.get('https://www.healthspace.ca/Clients/VIHA/VIHA_Website.nsf/Outbreak')
vi = BeautifulSoup(vi.text,'html5lib')
outbreaks = vi.find('td')

outbreaks.getText()



' Currently, there are no reported outbreaks in VIHA hospitals or long term care facilities.'

In [29]:
# VCH
# List of ongoing and recently ended outbreaks
from PyPDF2 import PdfFileReader
import io
import re

vch = requests.get('http://www.vch.ca/Documents/facility-outbreak-bulletin.pdf')

with io.BytesIO(vch.content) as f:
            pdf = PdfFileReader(f)
            information = pdf.getDocumentInfo()
            pages = pdf.getNumPages()
            for i in range(0, pages):
                page = pdf.getPage(i)
                page_content = page.extractText()           
                print(page_content)
                
spl = page_content.split('\n')
spldf = pd.Series(spl)
spldf.iloc[10:29]

Facility Outbreak Bulletin
This bulletin lists ongoing and recently ended outbreaks in licensed long-term and acute care
facilities throughout Vancouver Coastal Health, as of 12:46 PM, 18-Aug-2020
DISEASE
LOCATION
RESTRICTIONS
IMPOSED
RESTRICTIONS
LIFTED
FACILITY
COVID-19
4505 Valley Dr, Vancouver
14-Aug-20
Arbutus Care Centre
COVID-19
9020 Bridgeport Road, Richmond
05-Aug-20
18-Aug-20
Richmond Lions Manor - Bridgeport, 2nd Floor
COVID-19
803 West 12th Avenue, Vancouver
06-Aug-20
15-Aug-20
Joseph & Rosalie Segal Family Health Centre, 8th 
Floor
COVID-19
1081 Burrard Street, Vancouver
16-Jul-20
28-Jul-20
St. Paul's Hospital, NICU
COVID-19
7801 Argyle St, Vancouver
09-Jun-20
Holy Family Hospital, LTCF (Rehabilitation Unit 
declared over)
Page 1 of 1
Red text denotes updates from previously issued bulletin
Grey text indicates that restrictions have been lifted
Restriction Imposed: 
Restrictions Lifted: 
               Date which outbreak measures were introduced
            Date which out

10    COVID-19                                         
11    4505 Valley Dr, Vancouver                        
12    14-Aug-20                                        
13    Arbutus Care Centre                              
14    COVID-19                                         
15    9020 Bridgeport Road, Richmond                   
16    05-Aug-20                                        
17    18-Aug-20                                        
18    Richmond Lions Manor - Bridgeport, 2nd Floor     
19    COVID-19                                         
20    803 West 12th Avenue, Vancouver                  
21    06-Aug-20                                        
22    15-Aug-20                                        
23    Joseph & Rosalie Segal Family Health Centre, 8th 
24    Floor                                            
25    COVID-19                                         
26    1081 Burrard Street, Vancouver                   
27    16-Jul-20                                 

In [31]:
vch_outbreaks = pd.DataFrame({'info' : spldf.iloc[11:29]})
vch_outbreaks.groupby(vch_outbreaks.index // 5).agg('\n'.join)['info'].str.split('\n', expand=True)
# vch_outbreaks.rename(columns = {'address', 'restrictionsimposed', 'facility', 'outbreaktype'}, inplace = True)
vch_outbreaks

,info
11,"4505 Valley Dr, Vancouver"
12,14-Aug-20
13,Arbutus Care Centre
14,COVID-19
15,"9020 Bridgeport Road, Richmond"
16,05-Aug-20
17,18-Aug-20
18,"Richmond Lions Manor - Bridgeport, 2nd Floor"
19,COVID-19
20,"803 West 12th Avenue, Vancouver"


In [ ]:
# Fraser
# Outbreaks that are ended are removed
f = requests.get(r'https://www.fraserhealth.ca/patients-and-visitors/current-outbreaks#.XzGwMS0ZNQI')
fraser = BeautifulSoup(f.text,'html5lib')
table = fraser.find_all('td')

newlist = []

for td in table:
        try:
            if td.find('h5').getText() == '\xa0':
                # if no info is provided, it will mess up indexing
                pass
            elif td.find('h5').getText() == '':
                # if no info is provided, it will mess up indexing
                pass
            else:    
                newlist.append(td.find('h5').getText())
        except AttributeError:
            pass
        
            
        
df = pd.DataFrame(newlist)

fraser = df.groupby(df.index // 4).agg('\n'.join)[0].str.split('\n',expand=True)

fraser.rename(columns = {0:'datedeclared', 1:'facility', 2:'facilitytype', 3:'outbreaktype'}, inplace = True)

fraser['health_authority'] = 'fraser'

fraser

In [28]:
# IH
ih = requests.get('https://www.interiorhealth.ca/YourEnvironment/CommunicableDiseaseControl/Outbreaks/Pages/default.aspx')
ih = BeautifulSoup(ih.text,'html5lib')
outbreaks = ih.find_all('table')[7]

outbreaks.find('td').text

'There are no items to show in this view of the "Outbreaks" list.'